In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import math
import random
from scipy.optimize import root_scalar
from scipy.stats import truncnorm
import itertools
from itertools import permutations, combinations
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
#!pip install sklearn-contrib-py-earth
#from pyearth import Earth

In [3]:
# inputs: 
# number (what draft pick they were)
# position
# hs (high school indicator)
# fv (scouting value)
# risk (20/80 scale) 
# OPTIONAL- more advanced scouting metrics? Worried about overfitting, and 

# output: 
# pct of max signing bonus they received 
# maybe just predict bonus as well, if better performance...

# models: 
# regression, as we want this to be linear 
# interaction terms, second order terms, regularization 
# potentially MARS? 

In [60]:
# draft data from Baseball America 

draft_1 = pd.read_csv('data_ba - 2021.csv')
draft_2= pd.read_csv('data_ba - 2022.csv')
draft_3= pd.read_csv('data_ba - 2023.csv')
draft_4= pd.read_csv('data_ba - 2024.csv')

# scouting data from Fangrpahs 

fg_1 = pd.read_csv('data_fg - 2021.csv')
fg_2 = pd.read_csv('data_fg - 2022.csv')
fg_3 = pd.read_csv('data_fg - 2023.csv')
fg_4 = pd.read_csv('data_fg - 2024.csv')

In [115]:
# merging data: 

df_1 = pd.merge(draft_1, fg_1, how='inner', on='name')
df_2 = pd.merge(draft_2, fg_2, how='inner', on='name') 
df_3 = pd.merge(draft_3, fg_3, how='inner', on='name')
df_4 = pd.merge(draft_4, fg_4, how='inner', on='name') 

In [116]:
# combining data

training_data = pd.concat([df_1, df_2, df_3], ignore_index=True) # testing data is df_4, 2024

# before we get the x and y, we need to one hot encode the position variable: 

training_data = pd.get_dummies(training_data, columns=['position'],dtype=int)
df_4 = pd.get_dummies(df_4, columns=['position'],dtype=int)

# x and y from training_data, along with testing x and y from 2024
x = training_data[['number','position_1B', 'position_C', 'position_IF', 'position_LHP', 'position_RHP', 'position_SS', 'hs','fv','risk']]
y = training_data['pct_max_bonus']

x_test = df_4[['number','position_1B', 'position_C', 'position_IF', 'position_LHP', 'position_RHP', 'position_SS', 'hs','fv','risk']]
y_test = df_4['pct_max_bonus']

In [117]:
# writing the csvs out for use in the MARS R file 

training_data.to_csv('training_data')
df_4.to_csv('df_4')

In [65]:
# fit the linear regression model 

model = LinearRegression()

model.fit(x, y)

print("In-sample R-squared:", model.score(x,y)) # decent in sample R-squared of 0.72, especially considering the sample size of 437

print("Out-of-sample R-squared:", model.score(x_test,y_test)) # bad out of sample R-squared of 0.54, which makes sense. We are over fitting 

In-sample R-squared: 0.7195168859351506
Out-of-sample R-squared: 0.5379121064205752


In [68]:
# fit a linear regression model with squared and interaction terms

poly_feats = PolynomialFeatures(degree=2, include_bias=False)

model_2 = make_pipeline(poly_feats, LinearRegression())

model_2.fit(x, y)

print("In-sample R-squared:", model_2.score(x, y)) # Very strong 0.89 in sample!

print("Out-of-sample R-squared:", model_2.score(x_test, y_test)) # now we do a very strong 0.79 out of sample!

In-sample R-squared: 0.8913543810620272
Out-of-sample R-squared: 0.7975540054244079


In [130]:
# now, we do it with a Lasso regularizer term 

poly_feats = PolynomialFeatures(degree=2, include_bias=False)

scaler = StandardScaler()

lasso = Lasso(alpha=0.0001, max_iter=10000)

model_3 = make_pipeline(poly_feats,scaler, lasso)

model_3.fit(x, y)

print("In-sample R-squared:", model_3.score(x, y)) 

print("Out-of-sample R-squared:", model_3.score(x_test, y_test)) # OOS of 0.57... worse than full model, likely just use full model. 

In-sample R-squared: 0.8898580528881019
Out-of-sample R-squared: 0.8089494795608855


In [103]:
#now we try it with ridge regression 

poly_feats = PolynomialFeatures(degree=1, include_bias=False)

scaler = StandardScaler()

ridge = Ridge(alpha=.01)

model_4 = make_pipeline(poly_feats,scaler, ridge)

model_4.fit(x, y)

print("In-sample R-squared:", model_4.score(x, y)) 

print("Out-of-sample R-squared:", model_4.score(x_test, y_test)) # OOS of 0.54... similar to lasso 

In-sample R-squared: 0.7195168856738287
Out-of-sample R-squared: 0.537920011326827


In [110]:
# look at the features for model 2 (chosen model)

pd.set_option('display.max_rows', None) # so that we print everything

df = pd.DataFrame({'Feature': features, 'Coefficient': coefs})
df_sorted = df.reindex(df['Coefficient'].abs().sort_values(ascending=False).index)
print(df_sorted) # most important first 

# EDA results: high school 

                      Feature   Coefficient
59                       hs^2 -1.092830e-01
7                          hs -1.092830e-01
29               position_C^2 -9.670931e-02
2                  position_C -9.670931e-02
34              position_C hs -7.855607e-02
5                position_RHP -7.345817e-02
50             position_RHP^2 -7.345817e-02
6                 position_SS -6.503813e-02
55              position_SS^2 -6.503813e-02
52            position_RHP hs -5.283639e-02
44             position_LHP^2  5.241347e-02
4                position_LHP  5.241347e-02
20              position_1B^2  4.048519e-02
1                 position_1B  4.048519e-02
56             position_SS hs -3.571040e-02
41             position_IF hs -3.026973e-02
26             position_1B hs  2.610479e-02
8                          fv -2.548053e-02
47            position_LHP hs -2.051676e-02
37              position_IF^2  7.186360e-03
3                 position_IF  7.186360e-03
60                      hs fv  7

In [111]:
import itertools
import statsmodels.api as sm
import pandas as pd
import numpy as np

def best_subsets(x, y, max_features=None):
    if max_features is None:
        max_features = x.shape[1]

    results = []
    for k in range(1, max_features + 1):
        for combo in itertools.combinations(x.columns, k):
            X_subset = sm.add_constant(x[list(combo)])
            model = sm.OLS(y, X_subset).fit()
            results.append({
                'features': combo,
                'adj_r2': model.rsquared_adj,
                'model': model
            })
    
    # Get best model by adjusted R²
    best = max(results, key=lambda r: r['adj_r2'])
    return best

# Run best subsets
best_model_result = best_subsets(x, y)

# Print summary
print("Best subset of features:", best_model_result['features'])
print("Adjusted R²:", best_model_result['adj_r2'])
print(best_model_result['model'].summary())

# Get test data with the same best subset of features
X_test_subset = sm.add_constant(x_test[list(best_model_result['features'])])

# Predict on test data
y_pred = best_model_result['model'].predict(X_test_subset)

# Calculate out-of-sample R² manually
ss_res = np.sum((y_test - y_pred) ** 2)
ss_tot = np.sum((y_test - np.mean(y_test)) ** 2)
r2_out_of_sample = 1 - ss_res / ss_tot

print("Out-of-sample R-squared:", r2_out_of_sample)

Best subset of features: ('number', 'position_IF', 'hs', 'fv', 'risk')
Adjusted R²: 0.7146051264182003
                            OLS Regression Results                            
Dep. Variable:          pct_max_bonus   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     219.3
Date:                Tue, 17 Jun 2025   Prob (F-statistic):          5.43e-116
Time:                        23:54:48   Log-Likelihood:                 365.51
No. Observations:                 437   AIC:                            -719.0
Df Residuals:                     431   BIC:                            -694.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------